#Vacation Py

In [1]:
# Setup and Importing Dependencies

import gmaps
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import requests

In [2]:
# Imports and defines API Key

from api_keys import g_key

##Store WeatherPy results into DataFrame
* Loads the .csv exported in WeatherPy to a new DataFrame

In [3]:
vacation_py_df = pd.read_csv("../WeatherPy/Output_Data/City_Weather.csv")

# Prints DataFrame
vacation_py_df

,Unnamed: 0,City,Temperature,Humidity,Longitude,Latitude,Cloudiness,Country,Windspeed,Date
0,0,lugovoy,44.40,55,72.76,42.95,29,KZ,5.61,1603667993
1,1,yulara,66.20,30,130.99,-25.24,28,AU,20.80,1603668257
2,2,kapaa,82.99,74,-159.32,22.08,40,US,9.17,1603668207
3,3,villa carlos paz,64.99,62,-64.50,-31.42,65,AR,15.26,1603668259
4,4,klaksvik,42.80,87,-6.59,62.23,26,FO,6.93,1603668260
...,...,...,...,...,...,...,...,...,...,...
587,587,manado,82.40,78,124.85,1.49,40,ID,1.74,1603668396
588,588,mushie,72.68,95,16.90,-3.02,98,CD,1.83,1603668521
589,589,belaya gora,9.99,93,146.42,68.53,10,RU,6.73,1603668522
590,590,togur,26.76,98,82.83,58.36,99,RU,10.18,1603668523


##Humidity Heatmap
* Configures gmaps
* Uses the Latitude and Longitude as locations as Humidity as the weight.
* Adds Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [7]:
# Variables
Locations = vacation_py_df[["Latitude", "Longitude"]]
Humidity = vacation_py_df["Humidity"]
figure = gmaps.figure()

# Creates Heat Layer
Heatmap_Layer = gmaps.heatmap_layer(Locations, weights=Humidity,
                                   dissipating=False, max_intensity=10,
                                   point_radius=1)

# Adds Heatmap Layer and Displays Figure
figure.add_layer(Heatmap_Layer)
figure

Figure(layout=FigureLayout(height='420px'))

##Making a New DataFrame to fit my Weather Criteria
Narrows down the cities to fit the following weather conditions and drops any rows with null values:
* A max temperature lower than 70 but higher than 60
* Wind speed less than 5 mph
* Cloudiness less than 5.0

In [9]:
ideal_vacay_cities = vacation_py_df.loc[(vacation_py_df["Temperature"]>=60)\
                             &(vacation_py_df["Temperature"]<=70)\
                             &(vacation_py_df["Windspeed"]<=5)\
                             &(vacation_py_df["Cloudiness"]<=5)].dropna()
ideal_vacay_cities

,Unnamed: 0,City,Temperature,Humidity,Longitude,Latitude,Cloudiness,Country,Windspeed,Date
6,6,gebze,64.40,88,29.43,40.80,0,TR,3.36,1603668260
43,43,cape town,63.00,84,18.42,-33.93,4,ZA,4.00,1603668117
80,80,marzuq,62.10,31,46.47,14.40,0,YE,0.74,1603668026
86,86,abu samrah,64.71,27,37.18,35.30,2,SY,4.94,1603668294
170,170,vallenar,65.05,28,-70.76,-28.57,0,CL,3.09,1603668334
179,179,eureka,69.01,54,-124.16,40.80,0,US,3.00,1603668203
193,193,deolali,69.39,68,73.83,19.95,0,IN,3.09,1603668345
223,223,yalova,64.40,88,29.28,40.66,0,TR,3.36,1603668360
237,237,menongue,67.64,27,17.69,-14.66,0,AO,3.11,1603668366
257,257,chichibu,61.00,71,139.08,35.99,1,JP,1.72,1603668376


##Hotel Map
* Stores into variable named `hotel_df`.
* Adds a "Hotel Name" column to the DataFrame
* Sets the parameters to search for hotels with 5000 meters.
* Hits the Google Places API for each city's coordinates.
* Stores the first Hotel result into the DataFrame.
* Plots markers on top of the heatmap.

In [14]:
hotel_df = ideal_vacay_cities[["City","Country","Latitude", "Longitude"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
6,gebze,TR,40.80,29.43,
43,cape town,ZA,-33.93,18.42,
80,marzuq,YE,14.40,46.47,
86,abu samrah,SY,35.30,37.18,
170,vallenar,CL,-28.57,-70.76,
179,eureka,US,40.80,-124.16,
193,deolali,IN,19.95,73.83,
223,yalova,TR,40.66,29.28,
237,menongue,AO,-14.66,17.69,
257,chichibu,JP,35.99,139.08,


In [16]:
base = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {  
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
}


for index, row in hotel_df.iterrows():

    # Variables
    lat = row['Latitude']
    long = row['Longitude']

    # Add Location keyword and values to Params dictionary
    params['location'] = f'{lat},{long}'

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"The closest hotel located in {row['City']} is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Hotel not found... skipping.")
        
    print("------------")

Retrieving Results for Index 6: gebze.
The closest hotel located in gebze is Holiday Inn Gebze - Istanbul Asia.
------------
Retrieving Results for Index 43: cape town.
The closest hotel located in cape town is Southern Sun Waterfront Cape Town.
------------
Retrieving Results for Index 80: marzuq.
Hotel not found... skipping.
------------
Retrieving Results for Index 86: abu samrah.
Hotel not found... skipping.
------------
Retrieving Results for Index 170: vallenar.
The closest hotel located in vallenar is Humacao Bed & Breakfast.
------------
Retrieving Results for Index 179: eureka.
The closest hotel located in eureka is Eureka Inn Trademark By Wyndham.
------------
Retrieving Results for Index 193: deolali.
The closest hotel located in deolali is Quality Inn Regency.
------------
Retrieving Results for Index 223: yalova.
The closest hotel located in yalova is Şeref Hotel.
------------
Retrieving Results for Index 237: menongue.
The closest hotel located in menongue is Ritz Lauca.


In [17]:
# Displays updated DataFrame

hotel_df

,City,Country,Latitude,Longitude,Hotel Name
6,gebze,TR,40.80,29.43,Holiday Inn Gebze - Istanbul Asia
43,cape town,ZA,-33.93,18.42,Southern Sun Waterfront Cape Town
80,marzuq,YE,14.40,46.47,
86,abu samrah,SY,35.30,37.18,
170,vallenar,CL,-28.57,-70.76,Humacao Bed & Breakfast
179,eureka,US,40.80,-124.16,Eureka Inn Trademark By Wyndham
193,deolali,IN,19.95,73.83,Quality Inn Regency
223,yalova,TR,40.66,29.28,Şeref Hotel
237,menongue,AO,-14.66,17.69,Ritz Lauca
257,chichibu,JP,35.99,139.08,Hotel Route Inn Chichibu


In [18]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [19]:
# Creates marker layer
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Adds Marker Layer and Displays Figure
figure.add_layer(marker_layer)
figure

Figure(layout=FigureLayout(height='420px'))